In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Jaison\\Documents\\Workspace\\Main Projects\\End_to_end_Employee\\End_to_End_Implementation\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Jaison\\Documents\\Workspace\\Main Projects\\End_to_end_Employee\\End_to_End_Implementation'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from Employee_Attition_End_to_end_ML_project_with_MLflow.constants import *
from Employee_Attition_End_to_end_ML_project_with_MLflow.utils.common import read_yaml, create_directories

In [7]:
from Employee_Attition_End_to_end_ML_project_with_MLflow.constants  import CONFIG_FILE_PATH
CONFIG_FILE_PATH
#src\Employee_Attition_End_to_end_ML_project_with_MLflow\constants

WindowsPath('config/config.yaml')

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [12]:
import os
import urllib.request as request
import zipfile
from Employee_Attition_End_to_end_ML_project_with_MLflow import logger
from Employee_Attition_End_to_end_ML_project_with_MLflow.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-21 22:27:06,329: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-21 22:27:06,336: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-21 22:27:06,341: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-21 22:27:06,346: INFO: common: created directory at: artifacts]
[2023-09-21 22:27:06,351: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-21 22:27:08,818: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 51314
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f2b3128f4d6399600ef702969e1385f20f7a878888ac9d8e644e11bdc1d53c75"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5346:584F:63EF5A:7973AA:650CFB79
Accept-Ranges: bytes
Date: Fri, 22 S